# Erasmus Neural Networks
http://michalbereta.pl/nn
## Control tasks for Perceptron and Widrow-Hoff model


## Before you start

Exacute the examples.

Then, do the tasks and send back the notebook.

Change the name of this notebook according to the schema: {YourSurname}\_{YourFirstName}\_{OriginalFileName}.

Be sure to fill all places with "YOUR ANSWER HERE".

When ready, send the notebook, with all the necessary files zipped, to the teacher.

## What to do

- Fill the methods of the following classes with your implementation.

- Read the comments to properly implement methods.

- Avoid loops when possible. Use numpy operations on matrices and vectors, instead.

- Execute the test code.

- Compare the results with the expected results given.

- Do not change the testing code, just the implementation od classes.

### Task 1 - Online version of Perceptron learning

Expected test output:

`
Loading train data...
Train data:
Number of examples= 500
Number of inputs= 10
Initial number of errors= 225
Training...
End of training
Errors for train data after training= 0
Loading test data...
Test data:
Number of examples= 439
Number of inputs= 10
Calculating answers for test data...
Saving classifications for test data...
Checking test error...
Test errors= 2  ->  0.004555808656036446 %
`

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random

class PerceptronOnline:
    """
    This is a perceptron which can process one example at a time.
    Assumption: class label is given as {-1, 1}
    """
    def __init__(self, num_of_inputs):
        """Perceptron constructor"""
        self.w = None
        self.b = None
        self.input_num = num_of_inputs
        self.InitWeights()
        
    def InitWeights(self):
        """Initializes weights to random values"""
        self.w = -1 + 2*np.random.random_sample(self.input_num,)
        self.b = random.random()
        
    def Forward(self, x): 
        """Forward pass - calculate the output as {-1, 1} of the neuron for one example x"""
        y = np.dot(x, self.w) + self.b
        return 1 if y > 0 else -1
    
    def Update(self, x, d, eta):
        """Calculate the output for x (one example), compare with d and update the weights if necessary"""
        u = self.Forward(x)
        if u!=d:
            self.w += eta*x*d
            self.b += eta*1*d
            
    def Train(self, X, D, eta, epochs):
        """
        Train for the maximum number of epochs or until the classification error is 0
        X: matrix with examples, each examples as a row
        D: vector of correct class labels for examples in rows of X
        The update to the weights vector is done after processing each example
        """
        while epochs>0 and self.errors!=0:
            for i in range(len(X)):
                self.Update(X[i], D[i], eta)
            epochs -= 1
            
    def CalculateErrors(self, X, D):
        """Calculates the number of errors - missclassifications"""
        solution = 0
        self.errors = 0
        for i in range(len(D)):
            solution = self.Forward(X[i])
            if solution!=D[i]:
                self.errors += 1
        return self.errors
    
##############################################################################
#DO NOT CHANGE THE FOLLOWING CODE
##############################################################################
print('Loading train data...')
train_data = np.loadtxt('train10D.csv')
X = train_data[:,:-1]
D = train_data[:,-1]
num_of_inputs = X.shape[1]
print('Train data:')
print('Number of examples=',X.shape[0])
print('Number of inputs=',num_of_inputs)

perc = PerceptronOnline(num_of_inputs)
perc.InitWeights()

start_errors = perc.CalculateErrors(X,D)
print('Initial number of errors=',start_errors)

print('Training...')
max_epochs = 100
eta = 0.01
perc.Train(X, D, eta, max_epochs)
print('End of training')

train_errors = perc.CalculateErrors(X,D)
print('Errors for train data after training=',train_errors)

print('Loading test data...')
test_data = np.loadtxt('test10D.csv')
print('Test data:')
print('Number of examples=',test_data.shape[0])
print('Number of inputs=',test_data.shape[1])

print('Calculating answers for test data...')
test_ans = []
for x in test_data:
    test_ans.append ( perc.Forward(x) )
test_ans = np.array(test_ans)
print('Saving classifications for test data...')
np.savetxt('test_data_classifications_perconline.csv', test_ans)

print('Checking test error...')
true_test_labels = np.loadtxt('test10D_correct_ans.csv')
test_errors = (true_test_labels != test_ans).sum()
print('Test errors=',test_errors,' -> ',test_errors/float(test_data.shape[0]),'%')


Loading train data...
Train data:
Number of examples= 500
Number of inputs= 10
Initial number of errors= 198
Training...
End of training
Errors for train data after training= 0
Loading test data...
Test data:
Number of examples= 439
Number of inputs= 10
Calculating answers for test data...
Saving classifications for test data...
Checking test error...
Test errors= 0  ->  0.0 %


### Task 2 - Batch version of Perceptron learning

Expected test output:

`
Loading train data...
Train data:
Number of examples= 500
Number of inputs= 10
Initial number of errors= 271
Training...
End of training
Errors for train data after training= 0
Loading test data...
Test data:
Number of examples= 439
Number of inputs= 10
Calculating answers for test data...
Saving classifications for test data...
Checking test error...
Test errors= 0  ->  0.0 %
`

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random

class PerceptronBatch:
    """
    This is a perceptron which can process all examples at a time.
    Assumption: class label is given as {-1, 1}
    """
    def __init__(self, num_of_inputs):
        """Perceptron constructor"""
        self.w = None
        self.b = None
        self.input_num = num_of_inputs
        self.InitWeights()
        
    def InitWeights(self):
        """Initializes weights to random values"""
        self.w = -1 + 2*np.random.random_sample(self.input_num,)
        self.b = random.random()
        
    def Forward(self, X): 
        """
        Forward pass - calculate the output as a vector of {-1, 1} of the neuron for all examples in X
        X: matrix with examples as rows
        """
        y = np.dot(X, self.w) + self.b
        
        y[y>0] = 1
        y[y<0] = -1
        
        return y
        
    def Update(self, X, D, eta):
        """Calculate the output for all examples in X (as rows), compare with D and update the weights if necessary"""
        u = self.Forward(X)
    
        result = u!=D
        
        self.w += eta*(np.multiply(X[result], D[result, None])).sum(axis=0)
        self.b += eta*(D[result, None]).sum()
            
    def Train(self, X, D, eta, epochs):
        """
        Train for the maximum number of epochs or until the classification error is 0
        X: matrix with examples, each examples as a row
        D: vector of correct class labels for examples in rows of X
        The update to the weights vector is done once per epoch, based on all examples
        """
        while epochs>0 and self.errors!=0:
            self.Update(X, D, eta)
            epochs -= 1
            
    def CalculateErrors(self, X, D):
        """Calculates the number of errors - missclassifications"""
        solution = self.Forward(X)
        self.errors = np.sum(solution!=D)
        return self.errors
    
##############################################################################
#DO NOT CHANGE THE FOLLOWING CODE
##############################################################################
print('Loading train data...')
train_data = np.loadtxt('train10D.csv')
X = train_data[:,:-1]
D = train_data[:,-1]
num_of_inputs = X.shape[1]
print('Train data:')
print('Number of examples=',X.shape[0])
print('Number of inputs=',num_of_inputs)

perc = PerceptronBatch(num_of_inputs)
perc.InitWeights()

start_errors = perc.CalculateErrors(X,D)
print('Initial number of errors=',start_errors)

print('Training...')
max_epochs = 100
eta = 0.01
perc.Train(X, D, eta, max_epochs)
print('End of training')

train_errors = perc.CalculateErrors(X,D)
print('Errors for train data after training=',train_errors)

print('Loading test data...')
test_data = np.loadtxt('test10D.csv')
print('Test data:')
print('Number of examples=',test_data.shape[0])
print('Number of inputs=',test_data.shape[1])

print('Calculating answers for test data...')
test_ans = perc.Forward(test_data)
print('Saving classifications for test data...')
np.savetxt('test_data_classifications_percbatch.csv', test_ans)

print('Checking test error...')
true_test_labels = np.loadtxt('test10D_correct_ans.csv')
test_errors = (true_test_labels != test_ans).sum()
print('Test errors=',test_errors,' -> ',test_errors/float(test_data.shape[0]),'%')


Loading train data...
Train data:
Number of examples= 500
Number of inputs= 10
Initial number of errors= 229
Training...
End of training
Errors for train data after training= 0
Loading test data...
Test data:
Number of examples= 439
Number of inputs= 10
Calculating answers for test data...
Saving classifications for test data...
Checking test error...
Test errors= 1  ->  0.002277904328018223 %


### Task 3 - Online version of Widrow-Hoff learning

Expected test output:

---CLASSIFICATION PROBLEM---

Loading train data...

Train data:

Number of examples= 500

Number of inputs= 10

Initial number of errors= 241

Initial MSE= 1.5702372419231343

Training...

End of training

Errors for train data after training= 6

MSE for train data after training= 0.31192984053640277

Loading test data...

Test data:

Number of examples= 439

Number of inputs= 10

Calculating answers for test data...

Saving classifications for test data...

Checking test error...

Test errors= 6  ->  0.01366742596810934 %


---REGRESSION PROBLEM---

x= [-6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5 -1.  -0.5  0.   0.5
  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5]

Initial MSE= 7.177903123126157

Training for regression...

After training, training MSE= 0.04428786345738948

After training, testing MSE= 0.005203074366538246


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random

class WidrowHoffOnline:
    """
    This is a Widrow-Hoff model which can process one example at a time.
    Can be used for both classification and regression problems
    Assumption: class label is given as {-1, 1} in classification problems
    """
    def __init__(self, num_of_inputs):
        """Constructor"""
        self.w = None
        self.b = None
        self.input_num = num_of_inputs
        self.InitWeights()
        
    def InitWeights(self):
        """Initializes weights to random values"""
        self.w = -1 + 2*np.random.random_sample(self.input_num,)
        self.b = random.random()
        
    def Forward(self, x): 
        """Forward pass - calculate the output as a real value of the neuron for one example x"""
        y = np.dot(x, self.w) + self.b
        return y
    
    def ForwardClassify(self, x): 
        """
        Forward pass - calculate the output as {-1, 1} by comparing the real output value of the neuron with threshold 0; 
        for one example x
        """
        y = np.dot(x, self.w) + self.b
        return 1 if y > 0 else -1
    
    def Update(self, x, d, eta):
        """Calculate the output for x (one example), and update the weights"""
        y = self.Forward(x)
        
        self.w += eta*np.multiply((d-y), x)
        self.b += eta*1*(d-y)
        
        return y
           
    def Train(self, X, D, eta, epochs):
        """
        Train for the maximum number of epochs
        X: matrix with examples, each examples as a row
        D: vector of real values required for examples in rows of X 
        """
        for i in range(epochs):
            for j in range(len(X)):
                self.Update(X[j], D[j], eta)
        
    def CalculateErrors(self, X, D):
        """
        Calculates the number of errors - missclassifications;
        D - assumed to be {-1, 1} here
        """
        solution = 0
        self.errors = 0
        for i in range(len(D)):
            solution = self.ForwardClassify(X[i])
            if solution!=D[i]:
                self.errors += 1
        return self.errors
        
    def CalculateMSE(self, X, D):
        """
        Calculates the mean square error 
        D - assumed to be a vector of any real values here
        """
        Y = []
        for i in range(len(D)):
            Y.append(self.Forward(X[i]))
        mse = np.sqrt(((Y - D) * (Y - D)).sum()) / D.shape[0]
        return mse
    
##############################################################################
#DO NOT CHANGE THE FOLLOWING CODE
############################################################################## 
print('---CLASSIFICATION PROBLEM---')
print('Loading train data...')
train_data = np.loadtxt('train10D.csv')
X = train_data[:,:-1]
D = train_data[:,-1]
num_of_inputs = X.shape[1]
print('Train data:')
print('Number of examples=',X.shape[0])
print('Number of inputs=',num_of_inputs)

perc = WidrowHoffOnline(num_of_inputs)
perc.InitWeights()

start_errors = perc.CalculateErrors(X,D)
start_mse = perc.CalculateMSE(X,D)
print('Initial number of errors=',start_errors)
print('Initial MSE=',start_mse)

print('Training...')
max_epochs = 200
eta = 0.001
perc.Train(X, D, eta, max_epochs)
print('End of training')

train_errors = perc.CalculateErrors(X,D)
train_mse = perc.CalculateMSE(X,D)
print('Errors for train data after training=',train_errors)
print('MSE for train data after training=',train_mse)

print('Loading test data...')
test_data = np.loadtxt('test10D.csv')
print('Test data:')
print('Number of examples=',test_data.shape[0])
print('Number of inputs=',test_data.shape[1])

print('Calculating answers for test data...')
test_ans = []
for x in test_data:
    test_ans.append ( perc.ForwardClassify(x) )
test_ans = np.array(test_ans)
print('Saving classifications for test data...')
np.savetxt('test_data_classifications_whonline.csv', test_ans)

print('Checking test error...')
true_test_labels = np.loadtxt('test10D_correct_ans.csv')
test_errors = (true_test_labels != test_ans).sum()
print('Test errors=',test_errors,' -> ',test_errors/float(test_data.shape[0]),'%')


print()
print('---REGRESSION PROBLEM---')
xmin = -6
xmax = 6
x = np.arange(xmin, xmax, 0.5)
print ('x=',x)

#real values of unknown process
a = 0.6
b = -0.4
d = a*x + b

#training data with noise (e.g., measurement errors)
sigma = 0.2
tr_d = d + np.random.randn(len(d)) * sigma

x.shape = (x.shape[0], 1)

perc_reg = WidrowHoffOnline(1)
start_mse = perc_reg.CalculateMSE(x, tr_d)
print('Initial MSE=', start_mse)

print('Training for regression...')
eta = 0.01
max_epochs = 100
perc_reg.Train(x, tr_d, eta, max_epochs)

train_mse = perc_reg.CalculateMSE(x, tr_d)
print('After training, training MSE=', train_mse)

#test data 
x_test = np.arange(xmin, xmax, 0.3)
d_test = a*x_test + b
x_test.shape = (x_test.shape[0],1)

test_mse = perc_reg.CalculateMSE(x_test, d_test)
print('After training, testing MSE=', test_mse)


---CLASSIFICATION PROBLEM---
Loading train data...
Train data:
Number of examples= 500
Number of inputs= 10
Initial number of errors= 205
Initial MSE= 0.06645373008824731
Training...
End of training
Errors for train data after training= 6
MSE for train data after training= 0.02497718320933732
Loading test data...
Test data:
Number of examples= 439
Number of inputs= 10
Calculating answers for test data...
Saving classifications for test data...
Checking test error...
Test errors= 6  ->  0.01366742596810934 %

---REGRESSION PROBLEM---
x= [-6.  -5.5 -5.  -4.5 -4.  -3.5 -3.  -2.5 -2.  -1.5 -1.  -0.5  0.   0.5
  1.   1.5  2.   2.5  3.   3.5  4.   4.5  5.   5.5]
Initial MSE= 1.0810289588055786
Training for regression...
After training, training MSE= 0.0322178091042909
After training, testing MSE= 0.012058634977381393


### Task 4 - Batch version of Widrow-Hoff learning

Expected test output:

---CLASSIFICATION PROBLEM---

Loading train data...

Train data:

Number of examples= 500

Number of inputs= 10

Initial number of errors= 320

Initial MSE= 3.519674085434046

Training...

End of training

Errors for train data after training= 6

MSE for train data after training= 0.3119085202726676

Loading test data...

Test data:

Number of examples= 439

Number of inputs= 10

Calculating answers for test data...

Saving classifications for test data...

Checking test error...

Test errors= 6  ->  0.01366742596810934 %


---REGRESSION PROBLEM---

Initial MSE= 13.232938807228429

Training for regression...

After training, training MSE= 0.033557507870294406

After training, testing MSE= 0.0035392872344154926


In [43]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import random

class WidrowHoffBatch:
    """
    This is a WidrowHoff model which can process all examples at a time.
    Can be used for both classification and regression problems
    Assumption: class label is given as {-1, 1} in classification problems
    """
    def __init__(self, num_of_inputs):
        """Constructor"""
        self.w = None
        self.b = None
        self.input_num = num_of_inputs
        self.InitWeights()
        
    def InitWeights(self):
        """Initializes weights to random values"""
        self.w = -1 + 2*np.random.random_sample(self.input_num,)
        self.b = random.random()
        
    def Forward(self, X): 
        """
        Forward pass - calculate the output as a vector of real values of the neuron for all examples in X
        X: matrix with examples as rows
        """
        y = np.dot(X, self.w) + self.b
        return y
    
    def ForwardClassify(self, X): 
        """
        Forward pass - calculate the output as a vector of {-1, 1} by comparing the real output values of the neuron with threshold 0; 
        X: matrix with examples as rows
        """
        y = np.dot(X, self.w) + self.b
        y[y>0] = 1
        y[y<0] = -1
        return y
    
    def Update(self, X, D, eta):
        """Calculate the output for all examples in X (as rows), and update the weights """
        y = self.Forward(X)
                    
        #for i in range(len(X)):
        #    self.w += eta*np.multiply((D[i]-y[i]),X[i])
        #    self.b += eta*(D[i]-y[i])
        
        delta = D - y
        
        self.w = eta*np.multiply(delta,X)
        self.b = eta*delta
        
        return y
        
    def Train(self, X, D, eta, epochs):
        """
        Train for the maximum number of epochs
        X: matrix with examples, each examples as a row
        D: vector of real values required for examples in rows of X 
        """
        for i in range(epochs):
            self.Update(X, D, eta)
        
    def CalculateErrors(self, X, D):
        """
        Calculates the number of errors - missclassifications
        D - assumed to be {-1, 1} here
        """
        solution = self.ForwardClassify(X)
        self.errors = np.sum(solution!=D)
        return self.errors
    
    def CalculateMSE(self, X, D):
        """
        Calculates the mean square error 
        D - assumed to be a vector of any real values here
        """
        Y = self.Forward(X)
        mse = np.sqrt(((Y - D) * (Y - D)).sum()) / D.shape[0]
        return mse
    
##############################################################################
#DO NOT CHANGE THE FOLLOWING CODE
##############################################################################     
print('---CLASSIFICATION PROBLEM---')
print('Loading train data...')
train_data = np.loadtxt('train10D.csv')
X = train_data[:,:-1]
D = train_data[:,-1]
num_of_inputs = X.shape[1]
print('Train data:')
print('Number of examples=',X.shape[0])
print('Number of inputs=',num_of_inputs)

perc = WidrowHoffBatch(num_of_inputs)
perc.InitWeights()

start_errors = perc.CalculateErrors(X,D)
start_mse = perc.CalculateMSE(X,D)
print('Initial number of errors=',start_errors)
print('Initial MSE=',start_mse)

print('Training...')
max_epochs = 100
eta = 0.001
perc.Train(X, D, eta, max_epochs)
print('End of training')

train_errors = perc.CalculateErrors(X,D)
train_mse = perc.CalculateMSE(X,D)
print('Errors for train data after training=',train_errors)
print('MSE for train data after training=',train_mse)

print('Loading test data...')
test_data = np.loadtxt('test10D.csv')
print('Test data:')
print('Number of examples=',test_data.shape[0])
print('Number of inputs=',test_data.shape[1])

print('Calculating answers for test data...')
test_ans = perc.ForwardClassify(test_data)
print('Saving classifications for test data...')
np.savetxt('test_data_classifications_whbatch.csv', test_ans)

print('Checking test error...')
true_test_labels = np.loadtxt('test10D_correct_ans.csv')
test_errors = (true_test_labels != test_ans).sum()
print('Test errors=',test_errors,' -> ',test_errors/float(test_data.shape[0]),'%')

print()
print('---REGRESSION PROBLEM---')
xmin = -6
xmax = 6
x = np.arange(xmin, xmax, 0.5)

#real values of unknown process
a = 0.6
b = -0.4
d = a*x + b

#training data with noise (e.g., measurement errors)
sigma = 0.2
tr_d = d + np.random.randn(len(d)) * sigma

x.shape = (x.shape[0], 1)

perc_reg = WidrowHoffBatch(1)
start_mse = perc_reg.CalculateMSE(x, tr_d)
print('Initial MSE=', start_mse)

print('Training for regression...')
eta = 0.001
max_epochs = 100
perc_reg.Train(x, tr_d, eta, max_epochs)

train_mse = perc_reg.CalculateMSE(x, tr_d)
print('After training, training MSE=', train_mse)

#test data 
x_test = np.arange(xmin, xmax, 0.3)
d_test = a*x_test + b
x_test.shape = (x_test.shape[0],1)

test_mse = perc_reg.CalculateMSE(x_test, d_test)
print('After training, testing MSE=', test_mse)


---CLASSIFICATION PROBLEM---
Loading train data...
Train data:
Number of examples= 500
Number of inputs= 10
Initial number of errors= 185
Initial MSE= 0.05666872076129856
Training...


ValueError: operands could not be broadcast together with shapes (500,) (10,) 